In [1]:
from plotly import graph_objects as go
import plotly.express as px
import pandas as pd
import dash
from dash.dependencies import Input, Output
from dash import Dash, html, dcc 
from urllib.request import urlopen
import json
from plotly.subplots import make_subplots
import numpy as np

In [4]:

veganGreen = 'rgb(16,114,60)' # Light-green for the vegan color option !!! Change for real color
veggieGreen = 'rgb(140,190,84)' # Dark-green for the vegetarian color option !!! Change for real color

df_nameIndex = pd.read_csv("data/fv_data.csv",
                 dtype={"fips": str})

df_sim = pd.read_excel("data_sim.xlsx",
                 dtype={"fips": str})
#df = df.fillna(0) # replace NA values with 0
df_nameIndex = df_nameIndex.set_index("Navn")
#df["Candidate"] = [df['Navn'][i]+f" ({df['Parti'][i][:2]})" for i, x in enumerate(df["Navn"])]
#df_nameIndex = df.set_index("Candidate")
# Tilføjer en randomiseret kostkolonne
#df_nameIndex["Kost"] = list(np.random.randint(low=1, high=6,size=len(df_nameIndex)))
#kost_dict = {5: "Kødspiser", 4:"Fleksitar",3:"Pescetar", 2: "Vegetar",1:"Veganer", 6: "Ønsker ikke at svare"}
kost_color = {"Kødspiser":"red", "Fleksitar":"turquoise","Pescetar":"blue", "Vegetar":veggieGreen,"Veganer":veganGreen
              ,"Ønsker ikke at svare" : "grey"}
#df_nameIndex["Kost"] = [kost_dict[x] for x in df_nameIndex["Kost"]]
#df_nameIndex["Kost_color"] = [kost_color[x] for x in df_nameIndex["Kost"]]



# List of the five new columns
#q1Answers = ['Daginstitutioner','Hospitaler, psykiatrien','Plejehjem, plejecentre og offentlig madudbringning til ældre', 'Offentlige arbejdspladser', 'ALLE offentlige institutioner']
#Adding the five columns, if not allready added
#if q1Answers[0] not in df.columns:
#    df = df.reindex(columns = df.columns[0:5].tolist() + q1Answers + df.columns[5:].tolist())

# First value adds random boolean(0,1) to "Alle offentlige institutioner" column.
# Second value adds random boolean (0,1) to the remaining four columns,
# depending on the boolean from the "Alle offentlige institutioner" column.
#q1Questions = df.columns[4:9]
#for i in range(0,len(df)):
#    value = 1 if random() > 0.7 else 0
#    df.loc[i,q1Questions[4:5]] = value
#    for col in q1Questions[0:4]:
#        value2 = 1 if random() > 0.5 else 0
#        if value == 0:
#            df.loc[i,col] = value2
#        else:
#            df.loc[i,col] = 0
# Definitions from the main-file

# This codeblock contains the variables for the dash-board
#Style
textBlack = 'rgb(0,0,0)' #Black for text

H2Style = {"fontSize": "25px",
            "color": veganGreen,
            "text-align": "center",
            'background': 'white',
            'font-family': 'Calibri',
            "margin-top": "20px",
            "margin-bottom":'10px',
            "padding":"1.5%"}

pStyle = {'fontSize': '18px',
            'color': textBlack,
            'text-align': 'center',
            'background': 'white',
            'font-family': 'Calibri',
            'margin-top': '-40px',
            'margin-bottom':'1px',
            'padding':'1.5%'}

#Lists
parties = [] # !!! Add list according to values from survey
candidates = [] # !!! Add list according to values from survey
questions = df_nameIndex.columns[11:25] # !!! Add questions to this list
kommuneList = df_nameIndex["Storkreds"].unique()  # !!! change list according to values from survey


# Placeholder text
loremIpsum = "Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum."



In [7]:
value = df_nameIndex["Storkreds"][10]
storkreds = value
question = df_nameIndex.columns[15]

print(value,"\n",
      storkreds, "\n",
      question)

Vestjyllands Storkreds 
 Vestjyllands Storkreds 
 Det skal sikres, at myndighederne på altomkost.dk vejleder konstruktivt i plantebaseret kost, herunder også sikrer konstruktiv vejledning til veganske børnefamilier.


In [8]:
df_temp = df_nameIndex[df_nameIndex["Storkreds"]==value]
df_temp = df_temp.sort_values("Score")
df_temp

,Storkreds,Parti,Kost,Q1,Q1.1,Q1.2,Q1.3,Q1.4,Q1.5,Alle offentlige køkkener skal pålægges at reducere deres CO2-udslip fra mad og måltider med 25% i 2025.,...,"Der skal være en klimaafgift på alle fødevarer, der afspejler deres drivhusgasudledning.",Der skal være en klimaafgift specifikt på alle animalske fødevarer,"Finansielle institutioner (banker, pensionskasser og investeringsrådgivere, långivere etc.) skal pålægges hvert år at afrapportere, hvordan deres porteføljer, produkter og långivning bidrager til grøn omstilling af landbruget, herunder især fra animalsk til plantebaseret produktion.","Fonden for Plantebaserede Fødevarer skal have mindst 200 mio. kr. ekstra årligt udover de eksisterende 85 mio. kr. årligt, så niveauet i højere grad svarer til omfanget af den nødvendige omstilling.","Forskningsmidler indenfor fødevarer skal omlægges til i højere grad at fokusere på klimavenlige, plantebaserede fødevarer frem for animalske.","Den danske animalske landbrugsproduktion står for en stor andel af den samlede danske drivhusgasudledning. Derfor skal antallet af landbrugsdyr (svin, kyllinger, køer, osv.) mindskes til halvdelen af det nuværende antal i 2030.","Landmænd, der vil omlægge deres produktion til flere plantebaserede afgrøder til mennesker, skal kunne modtage gratis efteruddannelse.","Støtteordninger til fødevareerhvervet skal omlægges til i højere grad at støtte klimavenlige, plantebaserede fødevarer frem for animalske.",Score,Kost_color
Navn,,,,,,,,,,,,,,,,,,,,,
Erling Præstekjær,Vestjyllands Storkreds,O. Dansk Folkeparti,Ønsker ikke at svare,Ved ikke / har ikke taget stilling,Ved ikke / har ikke taget stilling,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,grey
Jakob Kjærsgaard,Vestjyllands Storkreds,O. Dansk Folkeparti,Ønsker ikke at svare,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,grey
Freya Luna Andersen,Vestjyllands Storkreds,M. Moderaterne,Ønsker ikke at svare,"Enig - det vil jeg støtte, men jeg vil ikke se...","Enig - det vil jeg støtte, men jeg vil ikke se...","Enig - det vil jeg støtte, men jeg vil ikke se...","Enig - det vil jeg støtte, men jeg vil ikke se...","Enig - det vil jeg støtte, men jeg vil ikke se...","Enig - det vil jeg støtte, men jeg vil ikke se...",1,...,0,0,0,0,0,0,0,0,1,grey
Allan Berg,Vestjyllands Storkreds,D. Nye Borgerlige,Kødspiser,Uenig - det vil jeg ikke støtte,Uenig - det vil jeg ikke støtte,Uenig - det vil jeg ikke støtte,Uenig - det vil jeg ikke støtte,Uenig - det vil jeg ikke støtte,Uenig - det vil jeg ikke støtte,0,...,0,0,0,0,0,0,0,0,2,red
Ib Bjerregaard,Vestjyllands Storkreds,V. Venstre,Kødspiser,Uenig - det vil jeg ikke støtte,"Enig - det vil jeg støtte, men jeg vil ikke se...",Enig - det vil jeg arbejde aktivt for,"Enig - det vil jeg støtte, men jeg vil ikke se...",Uenig - det vil jeg ikke støtte,Uenig - det vil jeg ikke støtte,0,...,0,0,0,0,0,0,0,0,2,red
Tenna Røberg,Vestjyllands Storkreds,D. Nye Borgerlige,Fleksitar,Uenig - det vil jeg ikke støtte,Uenig - det vil jeg ikke støtte,Uenig - det vil jeg ikke støtte,Uenig - det vil jeg ikke støtte,Uenig - det vil jeg ikke støtte,Uenig - det vil jeg ikke støtte,0,...,0,0,0,0,0,0,1,0,3,turquoise
Thorbjørn Jacobsen,Vestjyllands Storkreds,I. Liberal Alliance,Kødspiser,"Enig - det vil jeg støtte, men jeg vil ikke se...","Enig - det vil jeg støtte, men jeg vil ikke se...","Enig - det vil jeg støtte, men jeg vil ikke se...","Enig - det vil jeg støtte, men jeg vil ikke se...","Enig - det vil jeg støtte, men jeg vil ikke se...","Enig - det vil jeg støtte, men jeg vil ikke se...",1,...,0,0,0,0,1,0,0,1,4,red
Hans Christian Molbech,Vestjyllands Storkreds,Å. Alternativet,Ønsker ikke at svare,Enig - det vil jeg arbejde aktivt for,Enig - det vil jeg arbejde aktivt for,Enig - det vil jeg arbejde aktivt for,Enig - det vil jeg arbejde aktivt for,Enig - det vil jeg arbejde aktivt for,Enig - det vil jeg arbejde aktivt for,2,...,0,0,0,0,0,0,0,0,4,grey
Dina Raabjerg,Vestjyllands Storkreds,C. Det Konservative Folkeparti,Ønsker ikke at svare,"Enig - det vil jeg støtte, men j

In [13]:
len(df_temp)*2

42

In [19]:

fig = go.Figure()
df_temp = df_nameIndex[df_nameIndex["Storkreds"]==value]
df_temp = df_temp.sort_values("Score")
lollipopRange = len(df_temp)*2


for j, mean in enumerate(df_temp["Score"]):
    candidate = df_temp.index[j]
    fig.add_trace(go.Scatter(y=[j,j],x=[0,mean],
                             marker_size = [0,12],
                             marker_color = df_temp["Kost_color"][j],
                             line=go.scatter.Line(color=veggieGreen),
                             hovertext=[df_temp.loc[candidate]["Parti"],df_temp.loc[candidate]["Parti"]],
                             showlegend=False,
                             hoverinfo=["none","x+y+text"],

                             )
                  )


# Adding a hidden scatterplot to add a legend with the dietary choices of the candidates
for k, v in kost_color.items():
    fig.add_trace(go.Scatter(x=[0],y=[0],
                             marker_size = [0],
                             marker_color = v,
                             name=k
                            ))
fig.update_layout(legend= {'itemsizing': 'constant',
                           # "itemsymbol":"circle"
                           })

#    fig.add_hline(y=df_temp["Score"].mean(),
#            line_width=0.5,
#            line_dash="dash",
#            line_color=veggieGreen,
#            annotation_text="Kommune gennemsnit",
#            annotation_position="bottom right")



tickvals_ = list(range(len(df_temp)))
ticktext_ = list(df_temp.index)
fig.update_layout(
    yaxis = dict(
        tickmode = "array",
        tickvals = tickvals_,
        ticktext = ticktext_),
    title = {"text":f"Kandidater for {value}"}
    )

fig.show()

In [10]:

df_temp =  df_nameIndex[df_nameIndex["Storkreds"]==storkreds]
df_temp = df_temp.sort_values(question)

# Parents for sunburst
value_labels = {0:"Uenig", 1:"Delvist Enig", 2:"Enig"}
parents_candidates = pd.Series([value_labels[x] for x in df_temp[question]],
                              index =df_temp.index)

sunburst_parents = []
for answer in parents_candidates.unique():
    sunburst_parents.append("")
sunburst_parents.extend(parents_candidates)

# Names for sunburst
sunburst_names = []

inner_names = list(parents_candidates.value_counts(sort=False).index)
sunburst_names.extend(inner_names)

candidate_names = list(df_temp.index)
sunburst_names.extend(candidate_names)

# Values for sunburst
sunburst_values = []

inner_values = list(parents_candidates.value_counts(sort=False))
sunburst_values.extend(inner_values)

candidate_values = []
for candidate in list(df_temp.index):
    candidate_values.append(1)

sunburst_values.extend(candidate_values)

fig = go.Figure()

color_dict = { "Enig":'rgb(15,122,55)',"Delvist Enig": 'rgb(169,220,163)',"Uenig":'rgb(218,241,212)'}
colors_sunburst = {}
for answer in parents_candidates.unique():
    colors_sunburst[answer] = color_dict[answer]

data = dict(
    names=sunburst_names,
    parent=sunburst_parents,
    value=sunburst_values)

fig.add_trace(go.Sunburst(
    labels=data['names'],
    parents=data['parent'],
    values=data['value'],
    branchvalues="total",
    marker_colors = list(colors_sunburst.values()),
    insidetextorientation='radial',
    ))
fig.update_layout(
    autosize=False,
    width=1200,
    height=720)
